### Picture Taking and Panorama Creation
This file contains the procedures to rotate the robot 360 degrees and then to construct a panorama for use in the localization process.

Below are listed the necessary imports. 
- `cozmo` allows us to run instructions on the robot, specifically the rotation in this case
- `cv2` allows us to stitch the 36 images together
- `os` is used for help creating file paths for the 36 stored partial images and the final panorama

In [8]:
#from PIL import Image
import cozmo
from cozmo.util import degrees
import cv2
import os

#### Directory configuration

The following two cells define the file path where the 36 images are to be stored so as not to clutter the main directory with the script and Jupyter notebook files. We create a subfolder pictures where the images will be stored. The second cell actually creates the directory, if it hasn't already been created.

In [9]:
# File path configuration
parent_dir = "/home/bendurham441/Documents/cs371-cozmo-fourthhour"
path = os.path.join(parent_dir, "pictures")

In [10]:
# Only needs to be run once
try:
    os.mkdir(path)
except FileExistsError:
    print("Directory already exists")

Directory already exists


#### Picture Taking

The following block defines a function that rotates the robot in 10 degree increments, taking a picture each time. These images are stored in the previously specified directory.

From what we have gathered, a photo must be annotated using `annotate_image()` prior to using `convert()`. Only after these two functions have been invoked on the image can we save the image as a JPEG.

In [11]:
def take_pictures(robot: cozmo.robot.Robot):
  # Initial setup, generate repeatable head angle and lift height
  robot.say_text("Okay here we go").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True
  
  # Saves picture of what Cozmo sees every 10 degrees.
  for d in range(36):
    fileName = "takingpics" + str(d * 10) + ".jpeg"
 
    robot.turn_in_place(degrees(10)).wait_for_completed()
    
    latest_image = robot.world.latest_image
    # annotation seems to be important, we have had issues saving images if we don't annotate
    annotated = latest_image.annotate_image()

    if latest_image is not None:
      print("image = %s" % latest_image)
      converted = annotated.convert()
      file_path = os.path.join(path, fileName)
      converted.save(file_path, "JPEG", resolution=10)

In [12]:
cozmo.run_program(take_pictures)

/home/bendurham441/.local/lib/python3.8/site-packages/cozmo/event.py:488: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  return await asyncio.wait_for(f, timeout, loop=self._loop)
2023-04-21 16:12:59,318 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2023-04-21 16:12:59,318 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2023-04-21 16:12:59,321 cozmo.general INFO     Found robot id=1
2023-04-21 16:12:59,321 cozmo.general INFO     Found robot id=1
2023-04-21 16:12:59,331 cozmo.general INFO     Connected to Android device serial=89c6ff8d
2023-04-21 16:12:59,331 cozmo.general INFO     Connected to Android device serial=89c6ff8d
2023-04-21 16:12:59,466 cozmo.general INFO     Robot id=1 serial=4322be95 initialized OK
2023-04-21 16:12:59,466 cozmo.general INFO    

image = <cozmo.world.CameraImage object at 0x7f22201e2fd0>
image = <cozmo.world.CameraImage object at 0x7f22201e2a60>
image = <cozmo.world.CameraImage object at 0x7f2221228e50>
image = <cozmo.world.CameraImage object at 0x7f22201e26a0>
image = <cozmo.world.CameraImage object at 0x7f22201d9280>
image = <cozmo.world.CameraImage object at 0x7f2221228e50>
image = <cozmo.world.CameraImage object at 0x7f22211f8fa0>
image = <cozmo.world.CameraImage object at 0x7f22211daf40>
image = <cozmo.world.CameraImage object at 0x7f2221223a30>
image = <cozmo.world.CameraImage object at 0x7f2221223370>
image = <cozmo.world.CameraImage object at 0x7f22211f8fa0>
image = <cozmo.world.CameraImage object at 0x7f22201e2940>
image = <cozmo.world.CameraImage object at 0x7f2221223040>
image = <cozmo.world.CameraImage object at 0x7f22201e2d60>
image = <cozmo.world.CameraImage object at 0x7f2221228fd0>
image = <cozmo.world.CameraImage object at 0x7f22201e27c0>
image = <cozmo.world.CameraImage object at 0x7f22201e2a0

2023-04-21 16:13:36,351 cozmo.general INFO     Shutting down connection
2023-04-21 16:13:36,351 cozmo.general INFO     Shutting down connection
2023-04-21 16:13:36,354 cozmo.general INFO     Android serial=89c6ff8d disconnected.
2023-04-21 16:13:36,354 cozmo.general INFO     Android serial=89c6ff8d disconnected.


image = <cozmo.world.CameraImage object at 0x7f22201d9f10>


#### Stitching

The following must be run only after the `pictures/` directory has been populated by running `taking_pictures()`. This block uses `cv2` to stitch the images together.

##### Important observations:
- At least in our experience, the stitching process can be erratic. Sometimes it takes only a few seconds, but we have observed cases where it takes up to as much as 5 minutes. It appears to differ by the environment. `cv2` stitching seems to struggle when we provide it with images where there are items very close by. Maybe this can be altered by altering the image inputs (maybe change the degree increments in which we are taking pictures?)

In [13]:
def stitch():
  print("Stitching")
  # construct an array of images to pass to the stitcher
  images = []
  for i in range(36):
    # read images from the pictures folder
    file_path = os.path.join(path, 'takingpics' + str((i * 10)) + ".jpeg")
    curr_image = cv2.imread(file_path)
    images.append(curr_image)

  # Stitch the provided images together
  stitcher = cv2.Stitcher.create()
  ret,pano = stitcher.stitch(images)
  
  # Output the final image to Panorama.jpeg
  cv2.imwrite('Panorama.jpeg',pano)

In [14]:
stitch()

Stitching
